In [2]:
# Import some packages
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# User modules
import utils.data_processing as dp

# Some configs
warnings.filterwarnings('ignore')
ox.config(log_console=True, use_cache=True)
pd.set_option('display.max_columns', 200)
#%matplotlib inline

In [3]:
# Define bounding box (W, S, E, N) for the area of Wellington we're interested in
# Copied from http://boundingbox.klokantech.com/
general_bbox = [174.5813,-41.4552,175.0722,-41.1527]

# Separate out the bounding box list into 4 vertices. 
south = general_bbox[1]
west = general_bbox[0]
north = general_bbox[3]
east = general_bbox[2]

# Set OSM bounding box
osm_bbox = [south, west, north, east]

In [8]:
# What types of entitities do we want to get? Currently restricted to amenities
tags = ['commerical', "office", "retail", "industrial"]
objects = ['way'] # like way, node, relation

# Generate the query string
compactOverpassQLstring = dp.generate_overpass_query(tags, objects, osm_bbox)
# Get Data
osmdf = dp.get_osm_data(compactOverpassQLstring, osm_bbox)

In [40]:
osmdf_ways = osmdf.query('type == "way"')[['id', 'nodes', 'type']]
osmdf_nodes = osmdf.query('type == "node"')[['id', 'lat', 'lon']]
osmdf_ways.head()

,id,nodes,type
0,48029394,"[610689114, 610689115, 610689116, 610689117, 4...",way
1,138564440,"[1519419008, 1519419060, 1519419003, 151941905...",way
2,138564441,"[1519419011, 1519419036, 1519419051, 151941904...",way
3,138564442,"[1519419039, 2296869111, 2296869115, 229686911...",way
4,147999585,"[1611093082, 1611093081, 1611093080, 161109307...",way


In [41]:
osmdf_ways['nodes_clean'] = osmdf_ways['nodes'].apply(convert_list_string)
osmdf_ways_clean = osmdf_ways.set_index(['id', 'type'])['nodes_clean'].apply(pd.Series).stack().reset_index()
osmdf_ways_clean.columns = ['way_id', 'type', 'sample_num', 'nodes']

In [45]:
osmdf_clean = pd.merge(osmdf_ways_clean, osmdf_nodes, left_on='nodes', right_on='id').drop(['nodes'], axis=1)
osmdf_centroids = osmdf_clean.groupby('way_id').agg({'lat': 'mean', 'lon': 'mean' }).reset_index()

In [52]:
map_bldgs = folium.Map(location=[-41.29, 174.8], zoom_start = 11)
    
for each in osmdf_centroids.iterrows():
    folium.Marker(location = [each[1]['lat'],each[1]['lon']]).add_to(map_bldgs)

map_bldgs